In [28]:
import pandas as pd
import numpy as np
import scipy as sp
import collections
import gc
import pickle
import time
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Lambda, Flatten, Conv2DTranspose
from tensorflow.keras.layers import Dropout, GaussianNoise, Input, UpSampling2D
from tensorflow.keras.models import Model, Sequential, load_model

<h2>Approach 1: CNN-Encoder

In [66]:
data = pd.read_csv("./train.csv")
X1 = data[data.columns[np.vectorize(lambda s: "start" in s)(data.columns.values)]]
X1 = X1.values.reshape(data.shape[0],25,25,1)
delta = data["delta"].values
Y = data[data.columns[np.vectorize(lambda s: "stop" in s)(data.columns.values)]]
Y = Y.values.reshape(data.shape[0],25,25)
data.shape, X1.shape, delta.shape, Y.shape

((50000, 1252), (50000, 25, 25, 1), (50000,), (50000, 25, 25))

In [68]:
tf.keras.backend.clear_session()
deltainp = Input((2,), dtype=tf.int32) #Delta - Only in range 1-5
gofinp = Input((25,25,1)) #Game of Life Board
conv1 = Conv2D(filters=16, kernel_size=(2,2), strides=(1,1), padding="same", activation="relu")(gofinp)
max1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv1)
conv2 = Conv2D(filters=32, kernel_size=(2,2), strides=(1,1), padding="same", activation="relu")(max1)
max2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv2)
drp2 = Dropout(0.1)(max2)
conv3 = Conv2D(filters=64, kernel_size=(2,2), strides=(1,1), padding="same", activation="elu")(drp2)
max3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv3)
up1 = UpSampling2D(size=(2,2), interpolation="nearest")(max3)
dec1 = Conv2DTranspose(filters=32, strides=(1,1), kernel_size=(1,1), activation="elu")(up1)
up2 = UpSampling2D(size=(2,2), interpolation="nearest")(dec1)
dec2 = Conv2DTranspose(filters=16, strides=(1,1), kernel_size=(1,1), activation="relu")(up2)
up3 = UpSampling2D(size=(2,2), interpolation="nearest")(dec2)
dec3 = Conv2DTranspose(filters=5, strides=(1,1), kernel_size=(2,2), activation=None)(up3)
dec3 = tf.transpose(dec3, perm=[0,3,1,2])
final = tf.gather_nd(dec3, deltainp)
final = tf.keras.layers.Activation(activation="sigmoid")(final)
model = Model(inputs=[deltainp, gofinp], outputs=final)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-3), loss="mean_absolute_error")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 25, 25, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 25, 25, 16)   80          input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 12, 12, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 12, 12, 32)   2080        max_pooling2d[0][0]              
______________________________________________________________________________________________

In [59]:
def custom_fitter(model, train_indices):

In [58]:
x = tf.random.normal((2,25,25,5))
x = tf.transpose(x,perm=[0,3,1,2])

<h2> Approach 2: Stacked Encoder with Data Augmentation